# Rover Response Optimization

This notebook shows how variables in an fmdtools model can be optimized for resilience.

In [1]:
from fmdtools.sim.approach import SampleApproach
import fmdtools.analyze as an
import fmdtools.sim.propagate as prop
import fmdtools.sim.search as search
import matplotlib.pyplot as plt
import multiprocessing as mp
import time

Model is in defined rover_model.py

In [2]:
from rover_model import Rover, gen_params, plot_map, plot_trajectories

ModuleNotFoundError: No module named 'rover_model'

## Optimization

Here we define the optimization problem for the rover.

We use a parallel pool, staged execution, and minimal tracking options to lower computational cost as much as possible.

In [6]:
mdl = Rover()
mdl.modelparams['end_condition']=''
track={'functions':{"Environment":"in_bound"},'flows':{"Ground":"all"}}
rover_prob = search.ProblemInterface("rover_problem", mdl, pool=mp.Pool(5), staged=True, track=track)

In [7]:
mdl.params

Here we will be optimizing over faults in the drive system at 3 points during the drive simulation interval:

In [8]:
app_drive = SampleApproach(mdl, faults='Drive', phases={'global':[0,39]}, defaultsamp={'samp':'evenspacing','numpts':3})
rover_prob.add_simulation("drive_faults", "multi", app_drive.scenlist)

The variables are the correction factors int the fault management: 

In [9]:
rover_prob.add_variables("drive_faults", ("cor_f", (-10,100)), ("cor_d", (-100, 100)), ("cor_t", (-10,100)), vartype="param")

We can define multiple objectives, below we will use the end distance from find_classification. 

In [10]:
rover_prob.add_objectives("drive_faults", end_dist="end_dist", tot_deviation="tot_deviation")

Here we do some basic timing:

In [11]:
rover_prob.end_dist([0.5,0.5,0.5])
a=time.time()
rover_prob.end_dist([0.6,0.5,0.5])
t=time.time()-a

In [12]:
t

Alternatively, we can use:

In [13]:
rover_prob.time_sims([0.7,0.5,0.5])

In [14]:
rover_prob

### Rover Optimization: 

In [15]:
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch
import numpy as np

In [16]:
pymoo_prob = rover_prob.to_pymoo_problem(objectives="end_dist")

In [17]:
arg={}
pymoo_prob._evaluate([1,1,2],arg)

In [18]:
arg

In [19]:
algorithm=PatternSearch(x0=np.array([0,0,0])) 

In [31]:
res = minimize(pymoo_prob, algorithm, verbose=True)

In [32]:
res.X

### Results visualization

Here we look at the optimized results and compare with the starting results:

In [34]:
rover_prob.end_dist([   0.65790489, -100.        ,   -1.45828247])

In [35]:
fig = plot_trajectories(rover_prob._sims['drive_faults']['mdlhists'], app= app_drive)

In [39]:
an.plot.mdlhists(rover_prob._sims['drive_faults']['mdlhists'], fxnflowvals="Ground", time_slice=app_drive.times)

In [40]:
ecs = {k: v['endclass'] for k, v in rover_prob._sims['drive_faults']['results'].items()}
fig = an.plot.metric_dist(ecs, 'end_dist')

starting solution:

In [41]:
rover_prob.end_dist([0,0,0])

In [42]:
fig = plot_trajectories(rover_prob._sims['drive_faults']['mdlhists'], app= app_drive)

In [43]:
ecs = {k: v['endclass'] for k, v in rover_prob._sims['drive_faults']['results'].items()}
fig = an.plot.metric_dist(ecs, 'end_dist')

As shown, while the optimized correction factors don't mitigate all scenarios, they do increase the number of scenarios that are mitigated.